<a href="https://colab.research.google.com/github/alex-jk/buyers-ID-manual-project/blob/main/process_studies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook extracts and cleans text from PDF reports for use in training the buyer profile extractor model.**

**Add input and output examples to jsonl file**

In [1]:
!git clone https://github.com/alex-jk/buyers-ID-manual-project.git
%cd buyers-ID-manual-project

Cloning into 'buyers-ID-manual-project'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 40 (delta 13), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 2.79 MiB | 6.59 MiB/s, done.
Resolving deltas: 100% (13/13), done.
/content/buyers-ID-manual-project


In [2]:
# import json

# input_text = """
# Buyers were willing to travel significant distances to reach underage victims.
# One man cited the Atlanta airport as a regular meeting place.
# """

# output_text = """
# Buyers are often mobile and willing to travel, including to major hubs like the Atlanta airport.
# """

# entry = {"input": input_text.strip(), "output": output_text.strip()}

# with open("data/labeled_chunks.jsonl", "a", encoding="utf-8") as f:
#     f.write(json.dumps(entry) + "\n")

**Import libraries**

In [3]:
import pandas as pd
import json
import os

**Read input output file and write to jsonl**

In [4]:
input_output_df = pd.read_csv("data/buyers_id_manual_input_output.csv")

print(input_output_df.shape)
print(input_output_df.columns)
print("\n-------------------------------\n")
print(input_output_df.head())

(9, 3)
Index(['Study', 'Input', 'Output'], dtype='object')

-------------------------------

                                    Study  \
0  The Shapiro Group Georgia Demand Study   
1  The Shapiro Group Georgia Demand Study   
2  The Shapiro Group Georgia Demand Study   
3  The Shapiro Group Georgia Demand Study   
4  The Shapiro Group Georgia Demand Study   

                                               Input  \
0  Almost half these men are the age 30-39, with ...   
1  The data clearly debunk the myth that CSEC is ...   
2  Not only are 65% of men who buy sex with young...   
3  Craigslist is by far the most efficient medium...   
4  While many of the men who exploit these childr...   

                                              Output  
0  Almost half these men are the age 30-39, with ...  
1  Men who\nrespond to advertisements for sex wit...  
2  65% of men who buy sex with young females do s...  
3  Buyers respond to Craigslist ads 3 times more ...  
4  Nearly half of buyers

In [6]:
input_output_df = input_output_df.drop_duplicates(subset=["Study", "Input", "Output"])

# Convert to JSONL
with open("data/labeled_chunks.jsonl", "w") as f:
    for _, row in input_output_df.iterrows():
        json_obj = {
            "study": row["Study"],
            "input": row["Input"],
            "output": row["Output"]
        }
        f.write(json.dumps(json_obj) + "\n")

**Models for text extraction**

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [2]:
# --- Configuration ---
model_id = "microsoft/Phi-3-mini-4k-instruct"

# --- Load Model and Tokenizer ---
# Load the model with 4-bit quantization to save memory (requires bitsandbytes)
# Use device_map="auto" to automatically use GPU if available
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",         # Use GPU if available, otherwise CPU
        torch_dtype="auto",        # Automatically select appropriate dtype
        trust_remote_code=True,    # Phi-3 requires this
        # Optional: uncomment below for 4-bit loading (needs bitsandbytes)
        # load_in_4bit=True,
        # bnb_4bit_compute_dtype=torch.bfloat16
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    print(f"Model '{model_id}' loaded successfully.")

    # --- Create a Hugging Face Pipeline for easier text generation ---
    # Note: max_new_tokens controls how long the generated output can be. Adjust as needed.
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        # Adjust max_new_tokens if your extracted text might be longer
        # Needs to be long enough for the longest expected extraction + "NONE"
        max_new_tokens=256,
        # Temperature=0 means more deterministic output, higher means more creative/random
        temperature=0.0,
        # top_p=0.95, # Optional: nucleus sampling
        do_sample=False # Set to False for more deterministic extraction
    )

except Exception as e:
    print(f"Error loading model or creating pipeline: {e}")
    print("Ensure you have sufficient RAM/VRAM and necessary libraries installed.")
    print("Consider using Google Colab with a T4 GPU runtime.")
    # Exit if model loading fails
    exit()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Device set to use cuda:0


Model 'microsoft/Phi-3-mini-4k-instruct' loaded successfully.
